In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv("OR.csv")

In [3]:
data.head()

,X1,X2,Y
0,True,True,True
1,False,True,True
2,True,False,True
3,False,False,False


In [8]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['X1']=le.fit_transform(data['X1'])
data['X2']=le.fit_transform(data['X2'])
data['Y']=le.fit_transform(data['Y'])

data.head()

,X1,X2,Y
0,1,1,1
1,0,1,1
2,1,0,1
3,0,0,0


In [22]:
x=data[['X1','X2']]
y=data[['Y']]

print(x.shape)
print(y.shape)

(4, 2)
(4, 1)


In [58]:
# Define Entropy and Information Gain

def entropy(col):
    counts=np.unique(col,return_counts=True)
    N= float(col.shape[0])
    ent=0.0
    
    for i in counts[1]:
        p=i/N
        ent=ent+(-1.0*p*np.log2(p))
    
    return ent

In [59]:
col=np.array([1,1,1,0,0,0])
entropy(col)

1.0

In [60]:
def divide_data(x_data,fkey,fval):
    x_right=pd.DataFrame([],columns=x_data.columns)
    x_left=pd.DataFrame([],columns=x_data.columns)
        
    for ix in range(x_data.shape[0]):
        val=x_data[fkey].loc[ix]
        
        if val>fval:
            x_right=x_right.append(x_data.loc[ix])
        else:
            x_left=x_left.append(x_data.loc[ix])
    return x_left,x_right

In [61]:
x_left,x_right=divide_data(data,'X1',0.5)
print(x_left)
print(x_right)

  X1 X2  Y
1  0  1  1
3  0  0  0
  X1 X2  Y
0  1  1  1
2  1  0  1


In [66]:
def infoGain(x_data,fkey,fval):
    left,right=divide_data(x_data,fkey,fval)
    
    # % of samples are on left and right
    l=float(left.shape[0])/x_data.shape[0]
    r=float(right.shape[0])/x_data.shape[0]
    
    if left.shape[0]==0 or right.shape[0]==0:
        print("Reached Leaf Node")
        
    i_gain=entropy(x_data.Y)-(l*entropy(left.Y)+r*entropy(right.Y))
   
    return i_gain

In [67]:
## test our function

for fx in x.columns:
    print(fx)
    print(infoGain(data,fx,data[fx].mean()))

# Sor here sex has max info gain

X1
0.31127812445913283
X2
0.31127812445913283


In [64]:
## Decision Tree

class DecisionTree:
    
    #Constructor
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
        
    def train(self,X_train):
        features=['X1','X2']
        info_gains=[]
        
        for ix in features:
            i_gain=infoGain(X_train,ix,X_train[ix].mean())
            
            info_gains.append(i_gain)
            
        self.fkey=features[np.argmax(info_gains)]
        self.fval=X_train[self.fkey].mean()
        print("Making tree and feature is ",self.fkey)
        
        #Split
        data_left,data_right=divide_data(X_train,self.fkey,self.fval)
        data_left=data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
        
        
        if data_left.shape[0]==0 or data_right.shape[0]==0:
            if X_train.Y.mean()>=0.5:
                self.target=1
            else:
                self.target=0
            return
        
        if(self.depth>=self.max_depth):
            if X_train.Y.mean()>=0.5:
                self.target=1
            else:
                self.target=2
            return
        
        # recursive Case
        self.left=DecisionTree(depth=self.depth+2,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right=DecisionTree(depth=self.depth+2,max_depth=self.max_depth)
        self.right.train(data_right)
        
        #Setting target at every node
        if X_train.Y.mean()>=0.5:
            self.target=1
        else:
            self.target=0
        return
    
    
    def predict(self,test):
        if test[self.fkey]>self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)
        
        

In [65]:
# split=int(0.7*data_clean.shape[0])
train_data=data
# test_data=data_clean[split:]
# test_data=test_data.reset_index(drop=True)

dt=DecisionTree()
dt.train(train_data)

Gain ratio is 0.31127812445913283
Gain ratio is 0.31127812445913283
Making tree and feature is  X1
Reached Leaf Node
Gain ratio is 0.0
Gain ratio is 1.0
Making tree and feature is  X2
Reached Leaf Node
Gain ratio is 0.0
Reached Leaf Node
Gain ratio is 0.0
Making tree and feature is  X1
Reached Leaf Node
Gain ratio is 0.0
Reached Leaf Node
Gain ratio is 0.0
Making tree and feature is  X1
Reached Leaf Node
Gain ratio is 0.0
Gain ratio is 0.0
Making tree and feature is  X1
